# Categorizando Produtos

O nosso desafio:

Um problema comum enfrentado por e-commerce é a categorização de produtos. Em um site que adiciona novos produtos constantemente, cada um com sua própria descrição, é necessário classificar os itens em categorias, como eletrônicos, utensílios de cozinha ou livros.

Realizar essa categorização manualmente é impraticável devido ao grande volume de produtos. Portanto, a solução é utilizar um modelo que consiga classificar automaticamente a categoria de um produto com base na sua descrição.

## Modelo bart para a língua inglesa

In [1]:
from transformers import pipeline

In [2]:
classificador = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [3]:
categorias = ['electronics', 'food', 'toys', 'books']

In [ ]:
predicao = classificador('''Latest model of smartphone with 5G connectivity and 128GB internal storage''',
                         candidate_labels=categorias)

In [6]:
predicao

{'sequence': 'Latest model of smartphone with 5G connectivity and 128GB internal storage',
 'labels': ['electronics', 'toys', 'food', 'books'],
 'scores': [0.9613720774650574,
  0.013380033895373344,
  0.012821927666664124,
  0.012426052242517471]}

## Modelo bart para língua pontuguesa

link https://huggingface.co/knowledgator/comprehend_it-multilingual-t5-base

In [7]:
!pip install liqfit sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 833.3/833.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.8 MB/s eta 0:00:00


In [8]:
from liqfit.pipeline import ZeroShotClassificationPipeline
from liqfit.models import T5ForZeroShotClassification
from transformers import T5Tokenizer

model = T5ForZeroShotClassification.from_pretrained('knowledgator/comprehend_it-multilingual-t5-base')
tokenizer = T5Tokenizer.from_pretrained('knowledgator/comprehend_it-multilingual-t5-base')
classifier = ZeroShotClassificationPipeline(model=model, tokenizer=tokenizer,
                                                      hypothesis_template = '{}', encoder_decoder = True)


/usr/local/lib/python3.10/dist-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

You are using a model of type T5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [9]:
descricao = '''Este water gel leve e refrescante, proporciona hidratação imediata que ajuda a aliviar o repuxamento e aspereza da pele sensível.'''
categorias_candidatas = ['beleza', 'cozinha', 'livros']
resultado = classifier(descricao, categorias_candidatas, multi_label = False)
resultado

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'sequence': 'Este water gel leve e refrescante, proporciona hidratação imediata que ajuda a aliviar o repuxamento e aspereza da pele sensível.',
 'labels': ['beleza', 'livros', 'cozinha'],
 'scores': [0.9386342167854309, 0.04146720468997955, 0.01989862322807312]}

In [10]:
import pandas as pd
resultado = pd.DataFrame(resultado).drop(['sequence'], axis=1)
resultado

,labels,scores
0,beleza,0.938634
1,livros,0.041467
2,cozinha,0.019899


In [11]:
descricao = '''A fritadeira eletrica sem óleo start fry da elgin possui um design unico, capacidade para até 3,5 litros,
            potência de 1400w e revestimento antiaderente. Seu sistema de circulação de ar ultra rápido frita e economiza energia.
            Sua grelha de fritura é removível e super fácil de limpar. Ela conta com uma proteção contra super aquecimento.
            Possui controle de temperatura de 80°c a 200°c que permite você programar a temperatura de preparo para cada tipo de alimento,
            timer para até 60 minutos com aviso sonoro e desligamento automático, assim você pode deixar preparando sua refeição
            enquanto realiza outras tarefas.'''

categorias_candidatas = ['beleza', 'cozinha', 'livros']
resultado = classifier(descricao, categorias_candidatas, multi_label=False)
resultado = pd.DataFrame(resultado).drop(["sequence"], axis=1)
resultado

,labels,scores
0,cozinha,0.712628
1,beleza,0.236710
2,livros,0.050662


## Aplicando em vários dados

In [12]:
dados = pd.read_csv('https://raw.githubusercontent.com/alura-cursos/hugging_face/main/Dados/descricoes_produtos.csv')
dados

,Descrição
0,Liquidificador de alta potência com jarra de v...
1,"Forno Micro-ondas de 20 litros, com menu desco..."
2,Máquina de café espresso com reservatório de á...
3,Torradeira com capacidade para quatro fatias e...
4,"Panela elétrica multifuncional que cozinha, as..."
5,Smartphone com 128GB de armazenamento e câmera...
6,"Smartwatch monitoramento de passos, frequência..."
7,Notebook gamer com placa de vídeo potente e te...
8,"Smartphone com tela super retina XDR de 6,7 po..."
9,Câmera digital com zoom óptico de 20x e vídeos...


In [13]:
categorias = ['eletrodomésticos', 'eletrônicos', 'beleza', 'brinquedos']

def categorizar(descricao):
  resultado = classifier(descricao, categorias, multi_label=False)
  categoria_max = max(zip(resultado['labels'],resultado['scores']), key=lambda x: x[1])[0]
  return categoria_max

dados['Categoria'] = dados['Descrição'].apply(categorizar)

In [14]:
dados

,Descrição,Categoria
0,Liquidificador de alta potência com jarra de v...,eletrodomésticos
1,"Forno Micro-ondas de 20 litros, com menu desco...",eletrodomésticos
2,Máquina de café espresso com reservatório de á...,eletrodomésticos
3,Torradeira com capacidade para quatro fatias e...,eletrodomésticos
4,"Panela elétrica multifuncional que cozinha, as...",eletrodomésticos
5,Smartphone com 128GB de armazenamento e câmera...,eletrônicos
6,"Smartwatch monitoramento de passos, frequência...",eletrônicos
7,Notebook gamer com placa de vídeo potente e te...,eletrônicos
8,"Smartphone com tela super retina XDR de 6,7 po...",eletrônicos
9,Câmera digital com zoom óptico de 20x e vídeos...,eletrônicos
